In [1]:
from operator import itemgetter
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.messages import HumanMessage, AIMessage,SystemMessage, trim_messages
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


In [2]:
# 👇 Use your Windows IP here
llm = Ollama(
    model="llama3",
    base_url="http://192.168.1.3:11434"  # <- ← 🧠 IP address where ollama is running, given that we are using WSL.
)

/tmp/ipykernel_435465/1948476943.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(


In [3]:
trimmer = trim_messages(
    max_tokens=1024,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=True,
    start_on="human"
)

In [4]:
system_prompt = ("You are a helpful assistant. Your job is answering the user's questions very shortly.")
prompt = ChatPromptTemplate.from_messages([("system", system_prompt),MessagesPlaceholder(variable_name="messages")])

In [5]:
chain=RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer) | prompt | llm

In [14]:
message_history = [SystemMessage(content="You are a helpful assistant.")]
def conversation_flow(user_input):
    llm_generation = chain.invoke({"messages": message_history+[HumanMessage(content=user_input)]})
    message_history.append(HumanMessage(content=user_input))
    message_history.append(AIMessage(content=llm_generation))
    return llm_generation

In [22]:
message_history

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi, I am Kaleem', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello Kaleem! How can I assist you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I am a data scientist and I want to learn about MLops', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hi Kaleem! MLOps stands for Machine Learning Operations. It's the practice of operationalizing machine learning models, focusing on the development, testing, deployment, and maintenance of machine learning systems. MLOps aims to streamline the process, ensuring reproducibility, reliability, and efficiency. Would you like me to elaborate or provide some popular tools in this space?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Yes please!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='MLOps is indeed a crucial as

In [15]:
response = conversation_flow("Hi, I am Kaleem")
response

'Hello Kaleem! How can I assist you today?'